# 3D Semantic Map Building
In this notebook, we load the RGB-D images and poses generated from `Generate_trajectory.ipynb`, and build a 3D map

In [1]:
import numpy as np
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import open3d as o3d

from src.features.mapping import Geocentric3DMapBuilder
from src.config import default_map_builder_cfg, default_sim_cfg

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sim_cfg = default_sim_cfg()
map_builder_cfg = default_map_builder_cfg()
map_builder_cfg.NUM_SEMANTIC_CLASSES = 3
map_builder = Geocentric3DMapBuilder(map_builder_cfg, sim_cfg)


In [3]:
TRAJECTORY = "00001-UVdNNRcVyV1"
DEPTH_MAP_DIR = f"../data/interim/trajectories/train/{TRAJECTORY}/D"
RGB_IMAGE_DIR = f"../data/interim/trajectories/train/{TRAJECTORY}/RGB"
POSITIONS_FILE = f"../data/interim/trajectories/train/{TRAJECTORY}/positions.npy"
ROTATIONS_FILE = f"../data/interim/trajectories/train/{TRAJECTORY}/rotations.npy"

In [4]:
rotations = np.load(ROTATIONS_FILE).view(dtype=np.quaternion)
positions = np.load(POSITIONS_FILE)

map_builder.clear()
for i in tqdm(range(18)):
    depth_map = np.load(f"{DEPTH_MAP_DIR}/{i}.npy")
    rgb_image = cv2.imread(f"{RGB_IMAGE_DIR}/{i}.png")
    pose = (positions[i], rotations[i])
    map_builder.update_point_cloud(rgb_image, depth_map, pose)

100%|██████████| 18/18 [00:00<00:00, 91.86it/s] 


In [5]:
pcd = map_builder.point_cloud
o3d.visualization.draw_geometries([pcd])